## 第九章—树回归

### 1、复杂数据的局部性建模

### 2、连续和离散型特征的树的构建

In [ ]:
# CART算法代码
import numpy as np

def loadDataSet(fileName):      # general function to parse tab -delimited floats
    dataMat = []                # assume last column is target value
    fr = open(fileName)
    for line in fr.readlines():
        curLine = line.strip().split('\t')
        fltLine = list(map(float, curLine)) # map all elements to float()
        # fltLine = [float(i) for i in curLine]
        dataMat.append(fltLine)
    return dataMat

def binSplitDataSet(dataSet, feature, value):
    mat0 = dataSet[np.nonzero(dataSet[:,feature] > value)[0],:]
    mat1 = dataSet[np.nonzero(dataSet[:,feature] <= value)[0],:]
    return mat0, mat1

In [ ]:
myDat = loadDataSet('D:/data/study/AI/ML/MLcode/Ch09/ex00.txt')
myMat = np.mat(myDat)
# print(myMat[:,-1])

In [ ]:
testMat = np.mat(np.eye(4))
mat0, mat1 = binSplitDataSet(testMat, 1, 0.5)
print(mat0, '\n', mat1)

### 3、将CART算法用于回归

### 3.1、构建树

In [ ]:
# 回归树的切分函数
def regLeaf(dataSet):                   # returns the value used for each leaf
    return np.mean(dataSet[:,-1])

def regErr(dataSet):
    return np.var(dataSet[:,-1]) * np.shape(dataSet)[0]

def chooseBestSplit(dataSet, leafType = regLeaf, errType = regErr, ops = (1, 4)):
    tolS = ops[0]; tolN = ops[1]
    # if all the target variables are the same value: quit and return value
    if len(set(dataSet[:,-1].T.tolist()[0])) == 1:              # exit cond 1
        return None,leafType(dataSet)
    m, n = np.shape(dataSet)
    # the choice of the best feature is driven by Reduction in RSS error from mean
    S = errType(dataSet)
    bestS = np.inf; bestIndex = 0; bestValue = 0
    for featIndex in range(n-1):
        # 矩阵转化为集合
        # for splitVal in set(dataSet[:,featIndex]):
        for splitVal in set([feat[featIndex] for feat in dataSet.tolist()]):
            mat0, mat1 = binSplitDataSet(dataSet, featIndex, splitVal)
            if(np.shape(mat0)[0] < tolN) or (np.shape(mat1)[0] < tolN):
                continue
            newS = errType(mat0) + errType(mat1)
            if newS < bestS:
                bestIndex = featIndex
                bestValue = splitVal
                bestS = newS
    # if the decrease (S-bestS) is less than a threshold don't do the split
    if (S - bestS) < tolS:
        return None,leafType(dataSet)                            # exit cond 2
    mat0, mat1 = binSplitDataSet(dataSet, bestIndex, bestValue)
    if(np.shape(mat0)[0] < tolN) or (np.shape(mat1)[0] < tolN):  # exit cond 3
        return None, leafType(dataSet)
    return bestIndex, bestValue                                  # returns the best feature to split on
                                                                 # and the value used for that split

In [ ]:
def createTree(dataSet, leafType = regLeaf, errType = regErr, ops = (1, 4)):    # assume dataSet is NumPy Mat so we can array filtering
    feat, val = chooseBestSplit(dataSet, leafType, errType, ops)                # choose the best split
    if feat == None: return val                                                # if the splitting hit a stop condition return val
    retTree = {}
    retTree['spInd'] = feat
    retTree['spVal'] = val
    lSet, rSet = binSplitDataSet(dataSet, feat, val)
    retTree['left'] = createTree(lSet, leafType, errType, ops)
    retTree['right'] = createTree(rSet, leafType, errType, ops)
    return retTree

In [ ]:
myDat = loadDataSet('D:/data/study/AI/ML/MLcode/Ch09/ex00.txt')
myMat = np.mat(myDat)
#print(myMat)
print(createTree(myMat))

In [ ]:
myDat1 = loadDataSet('D:/data/study/AI/ML/MLcode/Ch09/ex0.txt')
myMat1 = np.mat(myDat1)
print(createTree(myMat1))

### 4、树剪枝

#### 4.1、预剪枝

In [ ]:
myDat2 = loadDataSet('D:/data/study/AI/ML/MLcode/Ch09/ex2.txt')
myMat2 = np.mat(myDat2)
print(createTree(myMat2))

In [ ]:
print(createTree(myMat2, ops = (10000, 4)))

#### 4.2、后剪枝

In [ ]:
# 回归树剪枝函数
def isTree(obj):
    return (type(obj).__name__=='dict')

def getMean(tree):
    if isTree(tree['right']): tree['right'] = getMean(tree['right'])
    if isTree(tree['left']): tree['left'] = getMean(tree['left'])
    return (tree['left'] + tree['right'])/2.0

def prune(tree, testData):
    if np.shape(testData)[0] == 0: return getMean(tree)        # if we have no test data collapse the tree
    if (isTree(tree['right']) or isTree(tree['left'])):        # if the branches are not trees try to prune them
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
    if isTree(tree['left']): tree['left'] = prune(tree['left'], lSet)
    if isTree(tree['right']): tree['right'] =  prune(tree['right'], rSet)
    # if they are now both leafs, see if we can merge them
    if not isTree(tree['left']) and not isTree(tree['right']):
        lSet, rSet = binSplitDataSet(testData, tree['spInd'], tree['spVal'])
        errorNoMerge = np.sum(np.power(lSet[:,-1] - tree['left'], 2)) + np.sum(np.power(rSet[:,-1] - tree['right'], 2))
        treeMean = (tree['left'] + tree['right']) / 2.0
        errorMerge = np.sum(np.power(testData[:,-1] - treeMean, 2))
        if errorMerge < errorNoMerge:
            print("merging")
            return treeMean
        else: return tree
    else: return tree

In [ ]:
myTree = createTree(myMat2, ops = (0, 1))
myDatTest = loadDataSet('D:/data/study/AI/ML/MLcode/Ch09/ex2test.txt')
myMat2Test = np.mat(myDatTest)
print(prune(myTree, myMat2Test))

### 5、模型树

In [ ]:
# 模型树的叶节点生成函数
def linearSolve(dataSet):               # helper function used in two places
    m,n = np.shape(dataSet)
    X = np.mat(np.ones((m, n)))
    Y = np.mat(np.ones((m, 1)))         # create a copy of data with 1 in 0th postion
    X[:,1:n] = dataSet[:,0:n-1]
    Y = dataSet[:,-1]                   # and strip out Y
    xTx = X.T * X
    if np.linalg.det(xTx) == 0.0:
        raise NameError('This matrix is singular, cannot do inverse,try increasing the second value of ops')
    ws = xTx.I * (X.T * Y)
    return ws, X, Y

def modelLeaf(dataSet):                 # create linear model and return coeficients
    ws, X, Y = linearSolve(dataSet)
    return ws

def modelErr(dataSet):
    ws, X, Y = linearSolve(dataSet)
    yHat = X * ws
    return np.sum(np.power(Y - yHat, 2))

In [ ]:
myMat2 = np.mat(loadDataSet('D:/data/study/AI/ML/MLcode/Ch09/exp2.txt'))
print(createTree(myMat2, modelLeaf, modelErr, (1,10)))

### 6、树回归与标准回归的比较

In [ ]:
# 用树回归进行预测
def regTreeEval(model, inDat):
    return float(model)

def modelTreeEval(model, inDat):
    n = np.shape(inDat)[1]
    X = np.mat(np.ones((1, n+1)))
    X[:,1:n+1] = inDat
    return float(X * model)

def treeForeCast(tree, inData, modelEval = regTreeEval):
    if not isTree(tree): return modelEval(tree, inData)
    if inData[tree['spInd']] > tree['spVal']:
        if isTree(tree['left']): return treeForeCast(tree['left'], inData, modelEval)
        else: return modelEval(tree['left'], inData)
    else:
        if isTree(tree['right']): return treeForeCast(tree['right'], inData, modelEval)
        else: return modelEval(tree['right'], inData)

def createForeCast(tree, testData, modelEval=regTreeEval):
    m = len(testData)
    yHat = np.mat(np.zeros((m, 1)))
    for i in range(m):
        yHat[i, 0] = treeForeCast(tree, np.mat(testData[i]), modelEval)
    return yHat

In [ ]:
trainMat = np.mat(loadDataSet('D:/data/study/AI/ML/MLcode/Ch09/bikeSpeedVsIq_train.txt'))
testMat = np.mat(loadDataSet('D:/data/study/AI/ML/MLcode/Ch09/bikeSpeedVsIq_test.txt'))
myTree = createTree(trainMat, ops = (1, 20))
yHat = createForeCast(myTree, testMat[:,0])
print(np.corrcoef(yHat, testMat[:,1], rowvar = 0)[0, 1])

In [ ]:
ws, X, Y = linearSolve(trainMat)
for i in range(np.shape(testMat)[0]):
    yHat[i] = testMat[i, 0]*ws[1, 0] + ws[0, 0]
print(np.corrcoef(yHat, testMat[:, 1], rowvar = 0)[0, 1])

### 7、使用Python的Tkinter库创建GUI

In [ ]:
import tkinter as tk
import matplotlib
matplotlib.use('TkAgg')
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure

In [ ]:
root = tk.Tk()
myLabel = tk.Label(root, text = 'Hello World')
myLabel.grid()
root.mainloop()

In [ ]:
# 用于构建树管理器界面的Tkinter小部件
def reDraw(tolS, tolN):
    reDraw.f.clf()                       # clear the figure
    reDraw.a = reDraw.f.add_subplot(111)
    if chkBtnVar.get():
        if tolN < 2: tolN = 2
        myTree = createTree(reDraw.rawDat, modelLeaf, modelErr, (tolS,tolN))
        yHat = createForeCast(myTree, reDraw.testDat, modelTreeEval)
    else:
        myTree = createTree(reDraw.rawDat, ops = (tolS, tolN))
        yHat = createForeCast(myTree, reDraw.testDat)
    reDraw.a.scatter(reDraw.rawDat[:, 0].tolist(), reDraw.rawDat[:, 1].tolist(), s = 5)     # use scatter for data set
    reDraw.a.plot(reDraw.testDat, yHat, linewidth = 2.0)                                    # use plot for yHat
    reDraw.canvas.show()

def drawNewTree():
    tolN,tolS = getInputs()             # get values from Entry boxes
    reDraw(tolS,tolN)

In [ ]:
# Matplotlib和Tkinter的代码集成
def getInputs():
    try: tolN = int(tolNentry.get())
    except: 
        tolN = 10 
        print("enter Integer for tolN")
        tolNentry.delete(0, END)
        tolNentry.insert(0,'10')
    try: tolS = float(tolSentry.get())
    except: 
        tolS = 1.0 
        print("enter Float for tolS")
        tolSentry.delete(0, END)
        tolSentry.insert(0,'1.0')
    return tolN,tolS

In [ ]:
root = tk.Tk()

tk.Label(root, text = "Plot Place Holder").grid(row = 0, columnspan = 3)

reDraw.f = Figure(figsize=(5,4), dpi=100) #create canvas
reDraw.canvas = FigureCanvasTkAgg(reDraw.f, master=root)
reDraw.canvas.show()
reDraw.canvas.get_tk_widget().grid(row=0, columnspan=3)

tk.Label(root, text = "tolN").grid(row = 1, column = 0)
tolNentry = tk.Entry(root)
tolNentry.grid(row = 1, column = 1)
tolNentry.insert(0, '10')
tk.Label(root, text = "tolS").grid(row = 2, column = 0)
tolSentry = tk.Entry(root)
tolSentry.grid(row = 2, column = 1)
tolSentry.insert(0, '1.0')
tk.Button(root, text = "ReDraw", command = drawNewTree).grid(row = 1, column = 2, rowspan = 3)
chkBtnVar = tk.IntVar()
chkBtn = tk.Checkbutton(root, text = "Model Tree", variable = chkBtnVar)
chkBtn.grid(row = 3, column = 0, columnspan = 2)

reDraw.rawDat = np.mat(loadDataSet('D:/data/study/AI/ML/MLcode/Ch09/sine.txt'))
reDraw.testDat = np.arange(min(reDraw.rawDat[:, 0]), max(reDraw.rawDat[:, 0]), 0.01)
reDraw(1.0, 10)
               
root.mainloop()

tk.Button(root, text = "Quit", fg = "black", command = root.quit).grid(row = 1, column = 2)